In [1]:
import rasterio
import pandas as pd
import numpy as np
from pathlib import Path

OUTPUT_DIR = Path('output')
INPUT_RASTER = OUTPUT_DIR / 'peta_kerawanan_purbalingga.tif'
OUTPUT_CSV = OUTPUT_DIR / 'prediksi_kerawanan_grid_lengkap.csv'


print(f"--- Skrip Ekstraksi Data Raster ke CSV Dimulai (Perbaikan Final) ---")
print(f"Membaca file: {INPUT_RASTER}")

try:
    with rasterio.open(INPUT_RASTER) as src:
        image = src.read(1)
        nodata_val = src.nodata if src.nodata is not None else np.nan
        transform = src.transform
        
        if np.isnan(nodata_val):
            rows, cols = np.where(~np.isnan(image))
        else:
            rows, cols = np.where(image != nodata_val)
        
        print(f"✓ Ditemukan {len(rows)} titik data (piksel) di dalam batas wilayah.") 
        
        values = image[rows, cols]
        lons, lats = rasterio.transform.xy(transform, rows, cols)

        print("Langkah 7: Membuat DataFrame...")
        df_pred = pd.DataFrame({
            'longitude': lons,
            'latitude': lats,
            'nilai_prediksi_kerawanan': values
        })


        df_pred.to_csv(OUTPUT_CSV, index=False, float_format='%.6f')
        print(f"✓ Data prediksi grid berhasil disimpan ke: {OUTPUT_CSV}")

      
        
        print("\n   STATISTIK DESKRIPTIF DATA PREDIKSI (untuk Tabel 4.8)")
        
        print(df_pred['nilai_prediksi_kerawanan'].describe().to_markdown(floatfmt=".6f"))
        # ==========================================================

        print("="*80)
        print(f"\n--- Selesai ---")

except FileNotFoundError:
    print(f"❌ ERROR: File {INPUT_RASTER} tidak ditemukan.")
    print("Pastikan Anda sudah menjalankan skrip interpolasi utama terlebih dahulu.")
except Exception as e:
    print(f"❌ Terjadi error saat mengekstraksi raster: {e}")

--- Skrip Ekstraksi Data Raster ke CSV Dimulai (Perbaikan Final) ---
Membaca file: output\peta_kerawanan_purbalingga.tif
✓ Ditemukan 67137 titik data (piksel) di dalam batas wilayah.
Langkah 7: Membuat DataFrame...
✓ Data prediksi grid berhasil disimpan ke: output\prediksi_kerawanan_grid_lengkap.csv

   STATISTIK DESKRIPTIF DATA PREDIKSI (untuk Tabel 4.8)
|       |   nilai_prediksi_kerawanan |
|:------|---------------------------:|
| count |               67137.000000 |
| mean  |                   1.214077 |
| std   |                   0.077141 |
| min   |                   1.040548 |
| 25%   |                   1.213998 |
| 50%   |                   1.214002 |
| 75%   |                   1.214002 |
| max   |                   2.462898 |

--- Selesai ---
